## Process an audio

In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")
audio_path = r"C:\Users\amoph\Desktop\speech_therapy\torgo_dataset\MC\MC01\Session1\wav_headMic\0194.wav"
waveform, sample_rate = torchaudio.load(audio_path)
waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)  # most models use 16kHz


C:\Users\amoph\AppData\Local\Temp\ipykernel_12352\3289419984.py:2: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


## Run through phoneme Model

In [4]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

processor = Wav2Vec2Processor.from_pretrained("excalibur12/wav2vec2-large-lv60_phoneme-timit_english_timit-4k")
model = Wav2Vec2ForCTC.from_pretrained("excalibur12/wav2vec2-large-lv60_phoneme-timit_english_timit-4k")

inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

probs = torch.nn.functional.softmax(logits, dim=-1)
pred_ids = torch.argmax(probs, dim=-1)
transcription = processor.batch_decode(pred_ids)[0]

print(transcription)

c:\Users\amoph\Documents\GitHub\dysarthria-mtl\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amoph\.cache\huggingface\hub\models--excalibur12--wav2vec2-large-lv60_phoneme-timit_english_timit-4k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


h# jh ey n m ey q er n m ao r m ah n iy bcl b ay w er kcl k ix ng hh aa r dcl h#
